# Neurosynth Compose Meta-Analysis Notebook

After you've built your meta-analysis on [neurosynth compose](https://compose.neurosynth.org), you can use this notebook to execute, upload, and explore the results.

In [ ]:
#@title 1) Enter Meta-Analysis ID Below: { vertical-output: true }
META_ID = "" #@param {type:"string"}
META_ID = ''.join(META_ID.split()) # preprocess string
if META_ID is None:
    raise Exception("UPDATE META_ID VARIABLE TO YOUR META-ANALYSIS")

### 2) Run All Cells to execute

Using the Toolbar (Runtime -> Run All) or Ctrl/⌘ + F9

---

In [ ]:
#@title Setup environment (~1 minute)
%%capture --no-display --no-stderr
!wget https://raw.githubusercontent.com/neurostuff/neurosynth-compose-notebook/main/requirements.txt
%pip install -r requirements.txt

In [ ]:
#@title Run Meta-Analysis (~1-10+ minutes)
from compose_runner.run import run  #  special workflow to run meta-analysis

url, meta_result = run(META_ID)

print(f"""
==================================================================================================
Your results are automatically uploaded to Neurosynth Compose. \n
View completed meta-analysis here: {url}
==================================================================================================""")

## Interactive Report

NiMARE can also generate a comprehensive report of the meta-analysis for you to view and explore. The report provides an overview of the input coordinates input, key statistical maps, statistically significant clusters, and diagnostics to view the contributions of individual studies to each cluster.


In [ ]:
from nimare.reports import run_reports
run_reports(meta_result, 'report')

In [ ]:
#@title View
import portpicker
import threading
import socket
import IPython

from six.moves import socketserver
from six.moves import SimpleHTTPServer

class V6Server(socketserver.TCPServer):
  address_family = socket.AF_INET6

class Handler(SimpleHTTPServer.SimpleHTTPRequestHandler):
  def end_headers(self):
        self.send_header('Access-Control-Allow-Origin', '*')
        super().end_headers()

port = portpicker.pick_unused_port()

def server_entry():
    httpd = V6Server(('::', port), Handler)
    # Handle a single request then exit the thread.
    httpd.serve_forever()

thread = threading.Thread(target=server_entry)
thread.start()

from google.colab import output
output.serve_kernel_port_as_iframe(port, path='/report/report.html')

In [ ]:
#@title Download archive
#@markdown This will create an archive and download the HTML report to your local environment
from google.colab import files
import shutil

folder_path = '/content/report'

# Zip the folder
shutil.make_archive('/content/report', 'zip', folder_path)



# Download the zip file
files.download('/content/report.zip')